In [ ]:
%matplotlib inline

In [ ]:
from sparapy.radiosity_fast import DRadiosityFast
from sparapy.radiosity import Radiosity
import sparapy.geometry as geo
import sparapy as sp
import cProfile
from datetime import datetime


sample_walls = [
    geo.Polygon(
        [[0, 0, 0], [1, 0, 0], [1, 0, 1], [0, 0, 1]],
        [1, 0, 0], [0, 1, 0]),
    geo.Polygon(
        [[0, 1, 0], [1, 1, 0], [1, 1, 1], [0, 1, 1]],
        [1, 0, 0], [0, -1, 0]),
    geo.Polygon(
        [[0, 0, 0], [1, 0, 0], [1, 1, 0], [0, 1, 0]],
        [1, 0, 0], [0, 0, 1]),
    geo.Polygon(
        [[0, 0, 1], [1, 0, 1], [1, 1, 1], [0, 1, 1]],
        [1, 0, 0], [0, 0, -1]),
    geo.Polygon(
        [[0, 0, 0], [0, 0, 1], [0, 1, 1], [0, 1, 0]],
        [0, 0, 1], [1, 0, 0]),
    geo.Polygon(
        [[1, 0, 0], [1, 0, 1], [1, 1, 1], [1, 1, 0]],
        [0, 0, 1], [-1, 0, 0]),
]




In [ ]:

cProfile.run('radiosity = DRadiosityFast.from_polygon(sample_walls, 0.005)', )

In [ ]:
# first call it need to compline, thats why its a bit slower
# and this is parallelized
start = datetime.now()
cProfile.run('(energy, distance) = radiosity.init_energy([0.5, 0.5, 0.5])', )
first_fast = datetime.now() - start

In [ ]:
# second time its very fast since its already compiled
# without numda it took 0.76s no its 0.030s on my machine
start = datetime.now()
cProfile.run('(energy, distance) = radiosity.init_energy([0.5, 0.5, 0.5])', )
second_fast = datetime.now() - start

In [ ]:
# Lets try the initial implementation
cProfile.run('radiosity_old = Radiosity(sample_walls, 0.005, 0, 1)')

In [ ]:
def init_energy(rad, source_pos):
    source = geo.SoundSource(source_pos, [1, 0, 0], [0, 0, 1])
    E_matrix = []
    for patches in rad.patch_list:
        patches.init_energy_exchange(
            rad.max_order_k, rad.ir_length_s, source,
            sampling_rate=rad.sampling_rate)
        E_matrix.append(patches.E_matrix)
    return E_matrix

start = datetime.now()
cProfile.run('E_matrix = init_energy(radiosity_old, [0.5, 0.5, 0.5])')
slow = datetime.now() - start

In [ ]:
print(f'number of patches: {radiosity.n_patches}')
print(f'{slow/first_fast} faster than the old implementation (first call)')
print(f'{slow/second_fast} faster than the old implementation (second call)')

In [ ]:
import numpy as np
max_sizes = np.array([0.5, 0.1, 0.05, 0.01, 0.005])
# max_sizes = np.array([0.1, 0.01])
fast_first = np.zeros_like(max_sizes)
fast_second = np.zeros_like(max_sizes)
slow = np.zeros_like(max_sizes)
number_of_patches = np.zeros_like(max_sizes)
repeat = 10
for i, max_size in enumerate(max_sizes):
    print(f'run: {max_size}')
    # run fast two times
    radiosity = DRadiosityFast.from_polygon(sample_walls, max_size)
    start = datetime.now()
    (energy, distance) = radiosity.init_energy([0.5, 0.5, 0.5])
    fast_first[i] = (datetime.now() - start).microseconds
    start = datetime.now()
    for _ in range(repeat):
        (energy, distance) = radiosity.init_energy([0.5, 0.5, 0.5])
    fast_second[i] = (datetime.now() - start).microseconds/repeat
    number_of_patches[i] = radiosity.n_patches
    # Run old
    radiosity_old = Radiosity(sample_walls, max_size, 0, 1)
    start = datetime.now()
    for _ in range(repeat):
        E_matrix = init_energy(radiosity_old, [0.5, 0.5, 0.5])
    slow[i] = (datetime.now() - start).microseconds/repeat

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
ax = plt.gca()
# ax.semilogy(number_of_patches, fast_first*1e-6, label='fast implementation (first call)')
ax.semilogy(number_of_patches, fast_second*1e-6, label='fast implementation (second call)', marker='o')
ax.semilogy(number_of_patches, slow*1e-6, label='old implementation (classes)', marker='o')
ax.grid()
ax.set_xscale('log')
ax.set_xlabel('number of patches')
ax.set_ylabel('time [s]')
ax.set_title(f'mean time to calculate initial energy (n={repeat})')
plt.legend()